In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We will be working with the same medical data used in the `pw` miniproject, but will be leveraging the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip
import os

In [6]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz') as f:
    scripts = pd.read_csv(f)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz',names=col_names, compression='gzip', header=None, sep=',', quotechar='"', error_bad_lines=False)
practices.columns

Index(['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code'], dtype='object')

In [8]:
with gzip.open('./dw-data/chem.csv.gz') as c:
    chem = pd.read_csv(c)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [9]:
def summaryStats(liste,index):
    
    total = 0 
    mean = 0 
    std = 0 
    q25 = 0 
    median = 0
    q75 = 0 
    
    
    total = sum(liste[index])
    mean = liste.describe()[index]["mean"]
    std = liste.describe()[index]["std"]
    q25 = liste.describe()[index]["25%"]
    q75 = liste.describe()[index]["75%"]
    median = liste[index].median()
    
    return (total,mean,std,q25,median,q75)
    
    

summary_stats = [('act_cost',summaryStats(scripts,"act_cost")),('items',summaryStats(scripts,"items")),('quantity',summaryStats(scripts,'quantity')),('nic',summaryStats(scripts,'nic'))]    


In [11]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [13]:
group = scripts.groupby('bnf_name').sum()[['items']]

dicti = {}

for i,k in enumerate(scripts['bnf_name']):
    
    if(scripts['bnf_name'][i] not in dicti):
        
        dicti[scripts['bnf_name'][i]] = scripts['items'][i] 
    
    else:
        dicti[scripts['bnf_name'][i]] += scripts['items'][i]






In [14]:
sorted_x = sorted(dicti.items(), key=lambda kv: kv[1])

In [15]:
grader.score.dw__most_common_item(most_common_item)

NameError: name 'most_common_item' is not defined

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [15]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [16]:
scripts.columns = ['code','bnf_code','bnf_name','items','nic','act_cost','quantity']
practices.sort_values(by=['post_code'], inplace = True)
practices.drop_duplicates(subset='code', keep = 'first',inplace=True) 
joined_merge = pd.merge(scripts, practices[['code','post_code']], how = 'inner', on = 'code')
joined = joined_merge.groupby(['post_code','bnf_name'], as_index=False)['items'].sum()
k = joined_merge.groupby(['post_code'], as_index=False)['items'].sum()
merged = pd.merge(joined, k, how = 'inner', on = 'post_code')
merged.sort_values(by=['items_x'], ascending = False, inplace = True)
x = merged.drop_duplicates(subset='post_code', keep = 'first')
x['items_percent'] = x['items_x'] / x['items_y']
x.sort_values(by=['post_code'], ascending = True, inplace = True )
x.drop(['items_x', 'items_y'], axis = 1, inplace = True )
items_by_region = [tuple(i) for i in x.values]  
items_by_region = items_by_region[:100]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [18]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [19]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [20]:
import math
import gzip
import pandas as pd
from static_grader import grader


chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
chem.head()
chem.columns

with gzip.open ( './dw-data/201701scripts_sample.csv.gz', 'rb' ) as f:
    scripts = pd.read_csv ( f )

with gzip.open ( './dw-data/practices.csv.gz', 'rb' ) as f:
    practices = pd.read_csv ( f )



practices.columns = ['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code'] 
practices = practices[['code', 'name']].sort_values (by = ['name'], ascending = True) 
practices = practices [~practices.duplicated(['code'])] 
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine'] 


check = '|'.join(opioids) 
chem_df1 = chem 
chem_df1 [ 'test' ] = chem_df1 [ 'NAME' ].apply ( lambda x: any ( [ k in x.lower() for k in opioids ] ) ) 
key2 = chem_df1 [ "test" ] == True 
chem_df1 = chem_df1 [ key2 ]  
chem_sub = list (chem_df1['CHEM SUB']) 


scripts['opioid'] = scripts [ 'bnf_code' ].apply(lambda x: 1 if x in chem_sub else 0)
std_devn = scripts.opioid.std ()
overall_rate = scripts.opioid.mean()

scripts = scripts.merge (practices, left_on = 'practice', right_on = 'code')
scripts['cnt'] = 0


opioids_per_practice = scripts.groupby ( [ 'practice', 'name' ], as_index = False ).agg ( { 'opioid': 'mean', 'cnt': 'count' } )
opioids_per_practice.drop_duplicates()

opioids_per_practice['opioid'] = opioids_per_practice ['opioid'] - overall_rate

opioids_per_practice['std_err'] = std_devn / opioids_per_practice['cnt'] ** 0.5
opioids_per_practice['z_score'] = opioids_per_practice['opioid'] / opioids_per_practice['std_err']

result = opioids_per_practice[['name', 'z_score', 'cnt']]


result.sort_values(by = 'z_score', ascending = False, inplace = True)
anomalies = [(k[1], k[2], k[3]) for k in result.itertuples()][:100]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [21]:
opioids_per_practice = ...

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [22]:
relative_opioids_per_practice = ...

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [23]:
standard_error_per_practice = ...
opioid_scores = ...

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [34]:
unique_practices = ...
anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [35]:
grader.score.dw__script_anomalies(anomalies)

Your score:  0.01


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [36]:
scripts16 = ...

In [37]:
import pandas as pd
from static_grader import grader

scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz')
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

scripts16_2 = scripts16[['bnf_name', 'items']]
scripts16_2.reset_index(inplace=True)
scripts16_2 = pd.DataFrame(scripts16_2.groupby('bnf_name').count())
scripts16_2.reset_index(inplace=True)

scripts_2=scripts[['bnf_name', 'items']]
scripts_2.reset_index(inplace=True)
scripts_2 = pd.DataFrame(scripts_2.groupby('bnf_name').count())
scripts_2.reset_index(inplace=True)

scripts16_4 = scripts16_2.merge(scripts_2, on='bnf_name')#, how="left"
scripts16_4.reset_index(inplace=True)
scripts16_4['items_y'].fillna(0, inplace=True)

scripts16_4['rate'] = (scripts16_4['items_y'] - scripts16_4['items_x'])/scripts16_4['items_x']
scripts16_4 = scripts16_4[scripts16_4['items_x'] > 49]

growth_rate = scripts16_4[['bnf_name','rate', 'items_x', ]] \
                                    .sort_values(by=['rate'], ascending=False) \
                                    .reset_index(drop=True)

top_50 = growth_rate.head(50)
bottom_50 = growth_rate.tail(50)
script_growth = [tuple(i) for i in pd.concat([top_50, bottom_50]).values]



In [38]:
script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [39]:
grader.score.dw__script_growth(script_growth)

Your score:  0.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [40]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [41]:
rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [42]:
relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [43]:
standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [44]:
rare_scores = ...

In [45]:
import math
import gzip
import numpy as np
import pandas as pd
from static_grader import grader

with gzip.open ( './dw-data/201701scripts_sample.csv.gz', 'rb' ) as f:
    scripts_all_columns = pd.read_csv ( f )

with gzip.open ( './dw-data/practices.csv.gz', 'rb' ) as f:
    practices = pd.read_csv ( f )

practices.columns = [ 'practice', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = practices[ [ 'practice', 'name' ] ].sort_values ( by = [ 'name' ], ascending = True)
practices = practices [ ~practices.duplicated ( [ 'practice' ] ) ]

scripts = scripts_all_columns [ [ 'practice', 'bnf_code', 'act_cost' ] ] [ : ]
total_script_count = len ( scripts )

scripts [ 'bnf_count' ] = 1
unique_bnf = scripts.groupby ( 'bnf_code', as_index = False ) [ 'bnf_count' ].count()
rare_threshold = 0.1 / len ( unique_bnf )

rare_cond = unique_bnf [ 'bnf_count' ] / total_script_count < rare_threshold

rare_bnf = unique_bnf [ rare_cond ] [ 'bnf_code' ]

practice_rare_cost = scripts [ scripts ['bnf_code'].isin ( rare_bnf ) ]\
                            .groupby ( 'practice', as_index = False ).agg ( { 'act_cost': 'sum' } )

practice_all_cost = scripts.groupby ( 'practice', as_index = False ).agg ( { 'act_cost': 'sum' } )

rare_cost_prop =  practice_all_cost.merge ( practice_rare_cost, on = 'practice', suffixes = [ '_all_bnf', '_rare_bnf' ] )
# rare_cost_prop = rare_cost_prop [ ~rare_cost_prop.duplicated ( [ 'practice' ] ) ]

rare_cost_prop [ 'rare_prop' ] = rare_cost_prop.apply ( lambda df: df.act_cost_rare_bnf / df.act_cost_all_bnf, axis = 1 )
# print(rare_cost_prop.head(100))

overall_rare_cost_prop = rare_cost_prop.act_cost_rare_bnf.sum() / rare_cost_prop.act_cost_all_bnf.sum()

rare_cost_prop = rare_cost_prop [ [ 'practice', 'rare_prop' ] ] [ : ]

rare_cost_prop [ 'relative' ] = rare_cost_prop.rare_prop - overall_rare_cost_prop
# print(rare_cost_prop)

std_error = rare_cost_prop.relative.std ()
# print(std_error)#0.06353378513687823

rare_cost_prop [ 'z_score' ] = rare_cost_prop.relative / std_error

rare_scores = rare_cost_prop.merge ( practices, on = 'practice' ) [ [ 'practice', 'name', 'z_score' ] ] [ : ]
rare_scores.sort_values ( by = 'z_score', ascending = False, inplace = True )
# print(rare_scores.head(100))

rare_scripts = [ ( k [ 1 ], k [ 2 ], k [ 3 ] ) for k in rare_scores.itertuples() ] [ : 100 ]
# rare_scripts

In [46]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*